# Process Louvain 3 Clusters
This notebook will load Louvain cluster data, process row/column names and generate [Clustergrammer](http://clustergrammer.readthedocs.io/getting_started.html) visualization [JSONs](http://clustergrammer.readthedocs.io/clustergrammer_js.html#visualization-json) to produce interactive heatmaps. 

In [1]:
import pandas as pd
from copy import deepcopy

from clustergrammer_widget import *
net = Network(clustergrammer_widget)

df = {}

In [2]:
df['lv3-ini'] = pd.read_csv('../data/MMC10_Li3_Tcell_liveCD45/Louvain_3/summary.csv', index_col=0)
df['lv3-ini'].head()

,Louvain_3,Treatment,N,Total,Freq,CD45,CD57,In115Di,IdU,CD103,...,CD38,CD95,LAG3,CXCR3,HLADR,TIGIT,GranzymeB,DNA,Osmium,CD11b
1,6,1_normal,8230,169733,0.048488,0.658259,0.169114,0.033096,0.240033,0.049980,...,0.142250,0.425633,0.091767,0.183889,0.122041,0.055854,0.435562,0.804734,0.862120,0.734667
2,6,2_border,250,55703,0.004488,0.690238,0.035653,0.012406,0.149922,0.040909,...,0.114886,0.371667,0.069548,0.203976,0.140674,0.046681,0.385222,0.814625,0.840464,0.764056
3,6,3_tumor,235,333723,0.000704,0.672674,0.034560,0.009759,0.216150,0.062991,...,0.158168,0.378995,0.079664,0.255813,0.148387,0.069659,0.409060,0.808674,0.823157,0.707447
4,7,1_normal,21617,169733,0.127359,0.429158,0.018248,0.005821,0.130442,0.052624,...,0.115924,0.278340,0.044500,0.088673,0.058971,0.042087,0.238030,0.751865,0.406691,0.569863
5,7,2_border,1015,55703,0.018222,0.531833,0.020442,0.006815,0.115594,0.047920,...,0.123638,0.322280,0.052708,0.097743,0.067522,0.038370,0.227413,0.767207,0.434562,0.678852


### Process Column Categories

In [3]:
cols = df['lv3-ini'].columns.tolist()
cluster_cats = {}
for inst_col in cols[:5]:

    inst_cats = df['lv3-ini'][inst_col].get_values().tolist()
    
    cluster_cats[inst_col] = inst_cats
    

In [4]:
rows = df['lv3-ini'].index.tolist()
print(len(rows))

87


In [5]:
new_rows = []
for inst_row in range(len(rows)):

    # gather category info for inst_row
    inst_dict = {}    

    inst_cats = cols[:5]
    
    inst_cats = [x for x in inst_cats if x != 'N' and x !='Total']
    
    for inst_cat_title in inst_cats:
            
        # inst_row is the index
        inst_cat = str(cluster_cats[inst_cat_title][inst_row])

        # add 'Cluster' to cluster index
        if 'Louvain_' in inst_cat_title:
            inst_cat = 'Cluster-' + inst_cat

        # remove number from 'treatment'
        if 'Treatment' in inst_cat_title:
            inst_cat = inst_cat.split('_')[1]

        # add to inst_dict
        inst_dict[inst_cat_title] = inst_cat
    
    # manually generate name
    inst_name = inst_dict[inst_cats[0]] + '_' + inst_dict[inst_cats[1]]
    
    inst_cat_name = inst_cat_title + ': ' + inst_cat
    
    inst_tuple = ('Name: ' + inst_name, )
    
    for inst_cat in inst_cats:

        inst_cat_name = inst_cat + ': ' + inst_dict[inst_cat]        
        
        inst_cat_name = inst_cat_name.replace('Treatment', 'Tissue-Type')
        inst_cat_name = inst_cat_name.replace('Louvain_3', 'Cluster')
        inst_cat_name = inst_cat_name.replace('Freq', 'Freq in tissue')
    
        inst_tuple = inst_tuple + (inst_cat_name, )
#     print(inst_tuple)
    
    new_rows.append(inst_tuple)

In [6]:
markers = cols[5:]
mat = df['lv3-ini'][markers].get_values()
mat.shape

(87, 39)

### Transpose matrix to get markers as rows

In [7]:
df['lv3-proc'] = pd.DataFrame(data=mat, columns=markers, index=new_rows).transpose()

### Remove In115Di Marker

In [8]:
print(df['lv3-proc'].shape)
df['lv3-proc'] = df['lv3-proc'].drop(['In115Di'])
print(df['lv3-proc'].shape)

(39, 87)
(38, 87)


# Louvain 3 Heatmap

In [9]:
net.load_df(df['lv3-proc'])
net.set_cat_color(axis='col', cat_index=2, cat_name='Tissue-Type: normal', inst_color='blue')
net.set_cat_color(axis='col', cat_index=2, cat_name='Tissue-Type: border', inst_color='red')
net.set_cat_color(axis='col', cat_index=2, cat_name='Tissue-Type: tumor', inst_color='yellow')
net.cluster(enrichrgram=False)
net.write_json_to_file(filename='../docs/json/LV3_clusters.json', net_type='viz')
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.


# Louvain 3 Heatmap Z-score Marker Levels

In [10]:
net.load_df(df['lv3-proc'])
net.normalize(axis='row', norm_type='zscore')
net.cluster(enrichrgram=False, views=[])
net.write_json_to_file(filename='../docs/json/LV3_clusters_marker-zscore.json', net_type='viz')
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.
